# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#import requests
import gmaps
import os
import json

# Import API key
from api_keys import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_df = pd.read_csv("../WeatherPy/weatherpy.csv")
weather_df


,City,Country,Lat,Lng,Cloudiness (%),Humidity (%),Max Temp (°F),Wind Speed (mph),Date
0,Cape Town,ZA,-33.9258,18.4232,75,92,54.12,5.75,1627592127
1,Salinópolis,BR,-0.6136,-47.3561,17,81,80.49,9.42,1627592613
2,George Town,MY,5.4112,100.3354,20,89,80.53,1.99,1627592390
3,Kedrovyy,RU,56.1667,91.8167,63,61,45.01,3.09,1627592615
4,Hithadhoo,MV,-0.6000,73.0833,100,71,82.49,6.17,1627592616
...,...,...,...,...,...,...,...,...,...
542,Działdowo,PL,53.2396,20.1700,40,67,62.37,8.12,1627593066
543,Urengoy,RU,65.9667,78.3667,64,95,54.32,12.59,1627593138
544,Sajanan,TN,37.0573,9.2426,0,49,81.99,1.77,1627593138
545,Uvat,RU,59.1433,68.8953,45,99,43.54,6.11,1627593140


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=gkey)

In [5]:
# Store latitude and longitude
location = weather_df[["Lat", "Lng"]]

# Store humidity
humidity = weather_df["Humidity (%)"]

In [6]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2.1)
max_intensity = np.max(humidity)

heat_layer = gmaps.heatmap_layer(location, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
new_weather_df = pd.DataFrame(weather_df, columns = ["City", "Max Temp (°F)", "Wind Speed (mph)", "Cloudiness (%)"])

max_temp = (new_weather_df["Max Temp (°F)"] <= 80) & (new_weather_df["Max Temp (°F)"] > 70)
wind_speed = (new_weather_df["Wind Speed (mph)"] < 10)
cloudiness = (new_weather_df["Cloudiness (%)"] == 0)

new_weather_df

,City,Max Temp (°F),Wind Speed (mph),Cloudiness (%)
0,Cape Town,54.12,5.75,75
1,Salinópolis,80.49,9.42,17
2,George Town,80.53,1.99,20
3,Kedrovyy,45.01,3.09,63
4,Hithadhoo,82.49,6.17,100
...,...,...,...,...
542,Działdowo,62.37,8.12,40
543,Urengoy,54.32,12.59,64
544,Sajanan,81.99,1.77,0
545,Uvat,43.54,6.11,45


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = pd.DataFrame(weather_df, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Cape Town,ZA,-33.9258,18.4232,
1,Salinópolis,BR,-0.6136,-47.3561,
2,George Town,MY,5.4112,100.3354,
3,Kedrovyy,RU,56.1667,91.8167,
4,Hithadhoo,MV,-0.6000,73.0833,
...,...,...,...,...,...
542,Działdowo,PL,53.2396,20.1700,
543,Urengoy,RU,65.9667,78.3667,
544,Sajanan,TN,37.0573,9.2426,
545,Uvat,RU,59.1433,68.8953,


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
marker_layer = gmaps.marker_layer(locations, info_box_content= hotel_info)

fig.add_layer(marker_layer)

fig


Figure(layout=FigureLayout(height='420px'))